# 分析解码图的拓扑结构

In [6]:
from epmld.benchmark import generate_detector_error_model
from epmld.contraction_strategy.dem_to_hypergraph import DetectorErrorModelHypergraph
from epmld.contraction_strategy.hypergraph_to_connectivity import ConnectivityGraph


related_path = "/home/normaluser/ck/epmld/data/external/epmld_experiment_data/epmld_paper_experiment/noise_varying_threshold/surface_code"

save_path = "/home/normaluser/ck/epmld/experiment/epmld_paper_experiment/figure/graph_topology"

code_tasks = ["surface_code:rotated_memory_z"]
# "surface_code:rotated_memory_x",

# distances = [3, 5, 7]
distances = [3]
probabilities = [10]

noise_models = ["si1000"]
have_stabilizers = [True, False]
# have_stabilizers = [False]

# decoder_methods = ["MWPM", "EAMLD"]

decoder_methods = ["EAMLD"]

print("d & r & syndrome with data qubits & #nodes & #hyperedges & max hyperedge size \\\\ \\hline")

for code_task in code_tasks:
    error_type = "Z" if "memory_z" in code_task else "X" if "memory_x" in code_task else "other"
    for have_stabilizer in have_stabilizers:
        for d in distances:
            for p in probabilities:
                rounds = [1, d]
                for r in rounds:
                    for noise_model in noise_models:
                        for decoder_method in decoder_methods:
                            # 获取对应的检测器错误率模型，用于构建解码器
                            if decoder_method == "MWPM":
                                dem = generate_detector_error_model(d = d, r=r, p=p, noise_model=noise_model,
                                                                    error_type=error_type, decomposed_error = True,
                                                                    related_path=related_path, have_stabilizer = have_stabilizer)
                            else:
                                dem = generate_detector_error_model(d = d, r=r, p=p, noise_model=noise_model,
                                                                    error_type=error_type, decomposed_error = False,
                                                                    related_path=related_path, have_stabilizer = have_stabilizer)
                            
                            hypergraph = DetectorErrorModelHypergraph(detector_error_model= dem, have_logical_observable=False)
                            # 计算超图的相关数据
                            nodes = hypergraph.get_nodes()
                            hyperedges = hypergraph.get_hyperedges()
                            weights = hypergraph.get_weights()
                            max_detectors_per_hyperedge = 0
                            # 遍历所有超边
                            for hyperedge in hyperedges:
                                # 统计当前超边连接的检测器数量
                                num_detectors = len(hyperedge)
                                # 更新最大连接检测器数量
                                max_detectors_per_hyperedge = max(max_detectors_per_hyperedge, num_detectors)
                            
                            print(f"{d} & {r} & {have_stabilizer} & {len(nodes)} & {len(hyperedges)} & {max_detectors_per_hyperedge} \\\\")
                            
                            # # 保存d=3,r=1和d=5,r=1的连通性图片为PDF
                            if d in [3,5] and r==1:
                                connectivity_graph = ConnectivityGraph()
                                connectivity_graph.hypergraph_to_connectivity_graph(hypergraph)
                                connectivity_graph.draw("spring", save_path + "/"+ f"Connectivity Graph - d is {d}, r is {r}, Syndrome with Data Qubits is {have_stabilizer}.pdf")
                                # connectivity_graph.draw("spring", save_path, f"Connectivity Graph - d is {d}, r is {r}, Syndrome with Data Qubits is {have_stabilizer}")

d & r & syndrome with data qubits & #nodes & #hyperedges & max hyperedge size \\ \hline
3 & 1 & True & 8 & 23 & 2 \\
3 & 3 & True & 24 & 219 & 4 \\
3 & 1 & False & 4 & 10 & 2 \\
3 & 3 & False & 20 & 196 & 4 \\


In [2]:



from epmld.benchmark import generate_qldpc_detector_error_model
from epmld.contraction_strategy.dem_to_hypergraph import DetectorErrorModelHypergraph
from epmld.contraction_strategy.hypergraph_to_connectivity import ConnectivityGraph


related_path = "/home/normaluser/ck/epmld/data/external/epmld_experiment_data/epmld_paper_experiment/noise_varying_threshold/qldpc_code"

save_path = "/home/normaluser/ck/epmld/experiment/epmld_paper_experiment/figure/graph_topology"

# code_tasks = ["surface_code:rotated_memory_z"]
code_tasks = [ "bivariate_bicycle_code:rotated_memory_z"]
# "bivariate_bicycle_code:rotated_memory_x",
# "surface_code:rotated_memory_x",

# nkds = [3, 5, 7]
nkds = [[72, 12, 6], [144, 12, 12]]
# distances = [5]
probabilities = [10]

noise_models = ["si1000"]
have_stabilizers = [True, False]

# decoder_methods = ["MWPM", "EAMLD"]

decoder_methods = ["EAMLD"]

print("nkd & r & syndrome with data qubits & #nodes & #hyperedges & max hyperedge size \\\\ \\hline")

for code_task in code_tasks:
    error_type = "Z" if "memory_z" in code_task else "X" if "memory_x" in code_task else "other"
    for have_stabilizer in have_stabilizers:
        for nkd in nkds:
            for p in probabilities:
                d = nkd[2]
                rounds = [1, d]
                for r in rounds:
                    for noise_model in noise_models:
                        for decoder_method in decoder_methods:
                            # 获取对应的检测器错误率模型，用于构建解码器
                            dem = generate_qldpc_detector_error_model(nkd=nkd, r=r, p=p, noise_model=noise_model, 
                                                                    error_type=error_type,related_path=related_path,
                                                                    have_stabilizer = have_stabilizer)
                            
                            hypergraph = DetectorErrorModelHypergraph(detector_error_model= dem, have_logical_observable=False)
                            # 计算超图的相关数据
                            nodes = hypergraph.get_nodes()
                            hyperedges = hypergraph.get_hyperedges()
                            weights = hypergraph.get_weights()
                            max_detectors_per_hyperedge = 0
                            # 遍历所有超边
                            for hyperedge in hyperedges:
                                # 统计当前超边连接的检测器数量
                                num_detectors = len(hyperedge)
                                # 更新最大连接检测器数量
                                max_detectors_per_hyperedge = max(max_detectors_per_hyperedge, num_detectors)
                            
                            print(f"{nkd} & {r} & {have_stabilizer} & {len(nodes)} & {len(hyperedges)} & {max_detectors_per_hyperedge} \\\\")
                            
                            # # 保存d=3,r=1和d=5,r=1的连通性图片为PDF
                            if nkd == [72, 12, 6] and r==1:
                                connectivity_graph = ConnectivityGraph()
                                connectivity_graph.hypergraph_to_connectivity_graph(hypergraph)
                                connectivity_graph.draw("spring", save_path + "/"+ f"Connectivity Graph - nkd is {nkd}, r is {r}, Syndrome with Data Qubits is {have_stabilizer}.pdf")
                                # connectivity_graph.draw("spring", save_path, f"")

nkd & r & syndrome with data qubits & #nodes & #hyperedges & max hyperedge size \\ \hline
[72, 12, 6] & 1 & True & 72 & 432 & 6 \\
[72, 12, 6] & 6 & True & 432 & 16164 & 9 \\
[144, 12, 12] & 1 & True & 144 & 864 & 6 \\
[144, 12, 12] & 12 & True & 1728 & 67752 & 9 \\
[72, 12, 6] & 1 & False & 36 & 332 & 3 \\
[72, 12, 6] & 6 & False & 396 & 15932 & 9 \\
[144, 12, 12] & 1 & False & 72 & 617 & 3 \\
[144, 12, 12] & 12 & False & 1656 & 67366 & 9 \\
